In [1]:
# Importing necessary libs
import argparse
import datetime
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import traceback

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.autonotebook import tqdm

from backbone import EfficientDetBackbone
from efficientdet.dataset import CocoDataset, Resizer, Normalizer, Augmenter, collater
from efficientdet.loss import FocalLoss
from utils.sync_batchnorm import patch_replication_callback
from utils.utils import replace_w_sync_bn, CustomDataParallel, get_last_weights, init_weights, boolean_string

# Defining training parameters and class list

In [2]:

num_gpus=1

# mean and std in RGB order
mean= [0.485, 0.456, 0.406]
std= [0.229, 0.224, 0.225]

# coco anchors
anchors_scales= '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios= '[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)]'

# classes in the dataset
obj_list = ['3m''abus','accenture','adidas','adidas1','adidas_text','airhawk','airness','aldi','aldi_text','alfaromeo','allett','allianz','allianz_text',
            'aluratek','aluratek_text','amazon','amcrest','amcrest_text','americanexpress','americanexpress_text','android','anz','anz_text',
            'apc','apecase','apple','aquapac_text','aral','armani','armitron','aspirin','asus','at_and_t','athalon','audi','audi_text',
            'axa','bacardi','bankofamerica','bankofamerica_text','barbie','barclays','base','basf','batman','bayer','bbc','bbva',
          'becks','bellataylor','bellodigital','bellodigital_text','bem','benrus','bershka','bfgoodrich','bik','bionade',
            'blackmores','blizzardentertainment','bmw','boeing','boeing_text','bosch','bosch_text','bottegaveneta','bridgestone','bridgestone_text','budweiser','budweiser_text','bulgari','burgerking',
            'burgerking_text','calvinklein','canon','carglass','carlsberg','carters','cartier','caterpillar','chanel','chanel_text','cheetos','chevrolet',
            'chevrolet_text','chevron','chickfila','chimay','chiquita','cisco','citi','citroen','citroen_text','coach','cocacola','coke','colgate','comedycentral','converse','corona','corona_text',
            'costa','costco','cpa_australia','cvs','cvspharmacy','danone','dexia','dhl','disney','doritos','drpepper','dunkindonuts','ebay','ec','erdinger','espn','esso','esso_text',
            'evernote','facebook','fedex','ferrari','firefox','firelli','fly_emirates','ford','fosters','fritolay','fritos',
            'gap','generalelectric','gildan','gillette','goodyear','google','gucci','guinness','hanes','head','head_text','heineken','heineken_text','heraldsun','hermes','hersheys',
            'hh','hisense','hm','homedepot','homedepot_text','honda','honda_text','hp','hsbc','hsbc_text','huawei','huawei_text','hyundai','hyundai_text','ibm','ikea','infiniti''infiniti_text',
            'intel','internetexplorer','jackinthebox','jacobscreek','jagermeister','jcrew','jello','johnnywalker','jurlique','kelloggs',
            'kfc','kia','kitkat','kodak','kraft','lacoste','lacoste_text','lamborghini','lays','lego','levis','lexus','lexus_text',
            'lg','londonunderground','loreal','lotto','luxottica','lv','marlboro','marlboro_fig','marlboro_text','maserati','mastercard','maxwellhouse',
            'maxxis','mccafe','mcdonalds','mcdonalds_text','medibank','mercedesbenz','mercedesbenz_text','michelin','microsoft','milka',
            'millerhighlife','mini','miraclewhip','mitsubishi','mk','mobil','motorola','mtv','nasa','nb','nbc','nescafe','netflix','nike',
            'nike_text','nintendo','nissan','nissan_text','nivea','northface','nvidia','obey','olympics','opel','optus','optus_yes','oracle','pampers','panasonic'
            ,'paulaner','pepsi','pepsi_text','pepsi_text1','philadelphia','philips','pizzahut','pizzahut_hut','planters','playstation',
            'poloralphlauren','porsche','porsche_text','prada','puma','puma_text','quick','rbc','recycling','redbull','redbull_text','reebok','reebok1','reebok_text',
            'reeses','renault','republican','rittersport','rolex','rolex_text','ruffles','samsung','santander','santander_text','sap','schwinn',
            'scion_text','sega','select','shell','shell_text','shell_text1','siemens','singha','skechers','sony','soundcloud','soundrop',
            'spar','spar_text','spiderman','sprite','standard_liege','starbucks','stellaartois','subaru','subway','sunchips',
            'superman','supreme','suzuki','t-mobile','tacobell','target','target_text','teslamotors','texaco','thomsonreuters',
            'tigerwash','timberland','tissot','tnt','tommyhilfiger','tostitos','total','toyota','toyota_text','tsingtao','twitter','umbro',
            'underarmour','unicef','uniqlo','uniqlo1','unitednations','ups','us_president','vaio','velveeta','venus','verizon',
            'verizon_text','visa','vodafone','volkswagen','volkswagen_text','volvo','walmart','walmart_text','warnerbros','wellsfargo',
            'wellsfargo_text','wii','williamhill','windows','wordpress','xbox','yahoo','yamaha','yonex','yonex_text','youtube','zara']

In [3]:
lr = 1e-4
num_epochs = 2
val_interval = 1
save_interval = 1000
es_min_delta = 0.1
es_patience = 2
load_weights = 'weights/efficientdet-d0_1_18000.pth' 
head_only = True

# Defining EfficientDet model and the loss function for training 

Also defining the training steps:

In [4]:
class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss


# Defining the training loop 

* Saving the model every 1000 steps and saving the best model every epoch
* Log losses after every batch
* Using EfficientNet model as backbone of convolutional layers

In [5]:
def train():

    if num_gpus == 0:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    else:
        torch.manual_seed(42)

    training_params = {'batch_size': 3,
                       'shuffle': True,
                       'drop_last': True,
                       'collate_fn': collater,
                       'num_workers': 0}

    val_params = {'batch_size':3,
                  'shuffle': False,
                  'drop_last': True,
                  'collate_fn': collater,
                  'num_workers':0}

    input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]
    training_set = CocoDataset(root_dir=r'C:\Users\Acer\Downloads\openlogo', set='JPEGImages',
                               transform=transforms.Compose([Normalizer(mean=mean, std=std),
                                                             Augmenter(),
                                                             Resizer(input_sizes[0])]))
    training_generator = DataLoader(training_set, **training_params)

    val_set = CocoDataset(root_dir=r'C:\Users\Acer\Downloads\openlogo', set='JPEGImages',
                          transform=transforms.Compose([Normalizer(mean=mean, std=std),
                                                        Resizer(input_sizes[0])]))
    val_generator = DataLoader(val_set, **val_params)

    model = EfficientDetBackbone(num_classes=len(obj_list)+1, compound_coef=0,
                                 ratios=eval(anchors_ratios), scales=eval(anchors_scales))

    # load last weights
    if load_weights is not None:
        if load_weights.endswith('.pth'):
            weights_path = load_weights
        try:
            last_step = int(os.path.basename(weights_path).split('_')[-1].split('.')[0])
        except:
            last_step = 0

        try:
            ret = model.load_state_dict(torch.load(weights_path), strict=False)
        except RuntimeError as e:
            print(f'[Warning] Ignoring {e}')
            print(
                '[Warning] Don\'t panic if you see this, this might be because you load a pretrained weights with different number of classes. The rest of the weights should be loaded already.')

        print(f'[Info] loaded weights: {os.path.basename(weights_path)}, resuming checkpoint from step: {last_step}')
    else:
        last_step = 0
        print('[Info] initializing weights...')
        init_weights(model)

    # freeze backbone if train head_only
    if head_only:
        def freeze_backbone(m):
            classname = m.__class__.__name__
            for ntl in ['EfficientNet', 'BiFPN']:
                if ntl in classname:
                    for p in m.parameters():
                        p.requires_grad = False

        model.apply(freeze_backbone)
        print('[Info] freezed backbone')

    # apply sync_bn when using multiple gpu and batch_size per gpu is lower than 4
    #  useful when gpu memory is limited.
    # because when bn is disable, the training will be very unstable or slow to converge,
    # apply sync_bn can solve it,
    # by packing all mini-batch across all gpus as one batch and normalize, then send it back to all gpus.
    # but it would also slow down the training by a little bit.
    if num_gpus > 1 and batch_size // num_gpus < 4:
        model.apply(replace_w_sync_bn)
        use_sync_bn = True
    else:
        use_sync_bn = False

    # warp the model with loss function, to reduce the memory usage on gpu0 and speedup
    model = ModelWithLoss(model, debug=False)

    if num_gpus > 0:
        model = model.cuda()

    optimizer = torch.optim.AdamW(model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
    
    epoch = 0
    best_loss = 1e3
    best_epoch = 0
    step = max(0, last_step)
    model.train()
    num_iter_per_epoch = len(training_generator)
    try:
        for epoch in range(num_epochs):
            last_epoch = step // num_iter_per_epoch
            if epoch < last_epoch:
                continue

            epoch_loss = []
            progress_bar = tqdm(training_generator)
            for iter, data in enumerate(progress_bar):
                if iter < step - last_epoch * num_iter_per_epoch:
                    progress_bar.update()
                    continue
                try:
                    imgs = data['img']
                    annot = data['annot']

                    if num_gpus == 1:
                        imgs = imgs.cuda()
                        annot = annot.cuda()

                    optimizer.zero_grad()
                    cls_loss, reg_loss = model(imgs, annot, obj_list=obj_list)
                    cls_loss = cls_loss.mean()
                    reg_loss = reg_loss.mean()

                    loss = cls_loss + reg_loss
                    if loss == 0 or not torch.isfinite(loss):
                        continue

                    loss.backward()
                    # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
                    optimizer.step()

                    epoch_loss.append(float(loss))

                    progress_bar.set_description(
                        'E: {}/{}. I: {}/{}. Cls: {:.5f}.\n Reg: {:.5f}.\n Total: {:.5f}'.format(
                            epoch, num_epochs, iter + 1, num_iter_per_epoch, cls_loss.item(),
                            reg_loss.item(), loss.item()))

                    # log learning_rate
                    current_lr = optimizer.param_groups[0]['lr']
                    step += 1

                    if step % save_interval == 0 and step > 0:
                        save_checkpoint(model, f'efficientdet-d{0}_{epoch}_{step}.pth')
                        print('checkpoint...')

                except Exception as e:
                    print('[Error]', traceback.format_exc())
                    print(e)
                    continue
            scheduler.step(np.mean(epoch_loss))
            count=0
            
            if epoch % val_interval == 0:
                model.eval()
                loss_regression_ls = []
                loss_classification_ls = []
                for iter, data in enumerate(val_generator):
                    count+=1
                    if count==1000:
                        break
                    with torch.no_grad():
                        imgs = data['img']
                        annot = data['annot']

                        if num_gpus == 1:
                            imgs = imgs.cuda()
                            annot = annot.cuda()

                        cls_loss, reg_loss = model(imgs, annot, obj_list=obj_list)
                        cls_loss = cls_loss.mean()
                        reg_loss = reg_loss.mean()

                        loss = cls_loss + reg_loss
                        if loss == 0 or not torch.isfinite(loss):
                            continue

                        loss_classification_ls.append(cls_loss.item())
                        loss_regression_ls.append(reg_loss.item())
                        count+=1
                cls_loss = np.mean(loss_classification_ls)
                reg_loss = np.mean(loss_regression_ls)
                loss = cls_loss + reg_loss

                print(
                    'Val. Epoch: {}/{}.\n Class loss: {:1.5f}.\n Regr loss: {:1.5f}.\n Total loss: {:1.5f}'.format(
                        epoch, num_epochs, cls_loss, reg_loss, loss))

                if loss + es_min_delta < best_loss:
                    best_loss = loss
                    best_epoch = epoch

                    save_checkpoint(model, f'efficientdet-d{0}_{epoch}_{step}.pth')

                model.train()

                # Early stopping
                if epoch - best_epoch > es_patience > 0:
                    print('[Info] Stop training at epoch {}. The lowest loss achieved is {}'.format(epoch, best_loss))
                    break
    except KeyboardInterrupt:
        save_checkpoint(model, f'efficientdet-d{0}_{epoch}_{step}.pth')


In [6]:
def save_checkpoint(model, name):
    torch.save(model.model.state_dict(), os.path.join('weights', name))

In [ ]:
train()

loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
loading annotations into memory...
Done (t=0.32s)
creating index...
index created!
[Info] loaded weights: efficientdet-d0_1_18000.pth, resuming checkpoint from step: 18000
[Info] freezed backbone
